In [14]:
import zipfile as zp
import pandas as pd
import os

file_list = []

#It is preferable to use extractall for a single file. zp.zipfile extracts to buffer.
with zp.ZipFile('dataPandas.zip','r') as dataPandas:
    for file in dataPandas.namelist():
        file_list.append(file)
print(file_list)

#The DataPandas in quotes is the name of a new folder that the zip is extracted to.
zp.ZipFile('dataPandas.zip').extractall("DataPandas")
#./ is used here to find a folder named DataPandas nested within a folder named DataPandas
os.listdir('./DataPandas')

#The '.' is for working directory.
filepath = './DataPandas/DataPandas'

#Goal: Take all json person info and place it into a single dataframe.



['dataPandas/3dhubs_suppliers.csv', 'dataPandas/3dhubs_suppliers_machines.csv', 'dataPandas/dataPandas/', 'dataPandas/dataPandas/3dhubs_suppliers.csv', 'dataPandas/dataPandas/3dhubs_suppliers_machines.csv', 'dataPandas/dataPandas/Sacramentorealestatetransactions.csv', 'dataPandas/dataPandas/SampleJSONfiles.json', 'dataPandas/dataPandas/SampleJSONfiles2.json', 'dataPandas/Sacramentorealestatetransactions.csv', 'dataPandas/SampleJSONfiles.json', 'dataPandas/SampleJSONfiles2.json']


In [19]:
import json
with open(filepath+'/SampleJSONfiles.json') as jfile:
    data = json.load(jfile)
    personDF = pd.DataFrame(data["users"])

personDF


,userId,firstName,lastName,phoneNumber,emailAddress
0,1,Binil,Starly,123456,bstarly@asu.edu
1,2,Julie,Swann,1293933456,jswann@asu.edu
2,3,Katniss,Everdeen,3939302,kniss@asu.edu
3,4,Ethan,Hunt,22940222,ehuntAwesome@asu.edu
4,5,Princess,Leia,1311111111,princessLeia@asu.edu
5,6,Bruce,Wayne,22340222,bwayne@asu.edu


In [20]:
import json
with open(filepath+'/SampleJSONfiles2.json') as jfile:
    data = json.load(jfile)
    personDF2 = pd.DataFrame(data["users"])

personDF2

,userId,firstName,lastName,phoneNumber,emailAddress
0,7,Bruce,Wayne,12900943,bwayne@asu.edu
1,8,Peter,Parker,1293456,pparker@asu.edu
2,9,Peter,Parker,1293456,pparker@asu.edu


Now the challenge is to merge both dataframes. 

They are of different sizes and no index has been set.

Also, one of the phoneNumber (s) is incorrect. 

Data cleaning must be done before we operate on the dataframe.

Combine them first...:

In [22]:
#Noticed that concat needs brackets within parentheses.
pd.concat([personDF, personDF2])



,userId,firstName,lastName,phoneNumber,emailAddress
0,1,Binil,Starly,123456,bstarly@asu.edu
1,2,Julie,Swann,1293933456,jswann@asu.edu
2,3,Katniss,Everdeen,3939302,kniss@asu.edu
3,4,Ethan,Hunt,22940222,ehuntAwesome@asu.edu
4,5,Princess,Leia,1311111111,princessLeia@asu.edu
5,6,Bruce,Wayne,22340222,bwayne@asu.edu
0,7,Bruce,Wayne,12900943,bwayne@asu.edu
1,8,Peter,Parker,1293456,pparker@asu.edu
2,9,Peter,Parker,1293456,pparker@asu.edu


Now we have some issues.

There are duplicates - some with different phoneNumber(s) and userIDs.

Our index stops at 5 and then resets to 0.

In [24]:
#Let's fix the index value now.

newDF = pd.concat([personDF, personDF2], ignore_index=True)
newDF

,userId,firstName,lastName,phoneNumber,emailAddress
0,1,Binil,Starly,123456,bstarly@asu.edu
1,2,Julie,Swann,1293933456,jswann@asu.edu
2,3,Katniss,Everdeen,3939302,kniss@asu.edu
3,4,Ethan,Hunt,22940222,ehuntAwesome@asu.edu
4,5,Princess,Leia,1311111111,princessLeia@asu.edu
5,6,Bruce,Wayne,22340222,bwayne@asu.edu
6,7,Bruce,Wayne,12900943,bwayne@asu.edu
7,8,Peter,Parker,1293456,pparker@asu.edu
8,9,Peter,Parker,1293456,pparker@asu.edu


Now let's drop duplicates.

We can use duplicate emails as a filter.

In [26]:
cleanDF = newDF.drop_duplicates(subset=["emailAddress"])
cleanDF

,userId,firstName,lastName,phoneNumber,emailAddress
0,1,Binil,Starly,123456,bstarly@asu.edu
1,2,Julie,Swann,1293933456,jswann@asu.edu
2,3,Katniss,Everdeen,3939302,kniss@asu.edu
3,4,Ethan,Hunt,22940222,ehuntAwesome@asu.edu
4,5,Princess,Leia,1311111111,princessLeia@asu.edu
5,6,Bruce,Wayne,22340222,bwayne@asu.edu
7,8,Peter,Parker,1293456,pparker@asu.edu


In [ ]:
#Missed this...
clean2DF = cleanDF.loc("")

In [36]:
content = pd.read_csv(filepath+'/Sacramentorealestatetransactions.csv', ',')

content.head()

#Can set street and city as a single index value. This is a composite key. 
# We know these are unique so we can use it.
records = content.set_index(['street', 'city'])
records
#Always set indices and do not rely on original one.

c:\Users\WTCra\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


,,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
street,city,,,,,,,,,,
3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,Wed May 21 00:00:00 EDT 2008,59222,38.631913,-121.434879
51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,Wed May 21 00:00:00 EDT 2008,68212,38.478902,-121.431028
2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,Wed May 21 00:00:00 EDT 2008,68880,38.618305,-121.443839
2805 JANETTE WAY,SACRAMENTO,95815,CA,2,1,852,Residential,Wed May 21 00:00:00 EDT 2008,69307,38.616835,-121.439146
6001 MCMAHON DR,SACRAMENTO,95824,CA,2,1,797,Residential,Wed May 21 00:00:00 EDT 2008,81900,38.519470,-121.435768
...,...,...,...,...,...,...,...,...,...,...,...
9169 GARLINGTON CT,SACRAMENTO,95829,CA,4,3,2280,Residential,Thu May 15 00:00:00 EDT 2008,232425,38.457679,-121.359620
6932 RUSKUT WAY,SACRAMENTO,95823,CA,3,2,1477,Residential,Thu May 15 00:00:00 EDT 2008,234000,38.499893,-121.458890
7933 DAFFODIL WAY,CITRUS HEIGHTS,95610,CA,3,2,1216,Residential,Thu May 15 00:00:00 EDT 2008,235000,38.708824,-121.256803


In [41]:
records.loc['51 OMAHA CT', 'SACRAMENTO']

#.loc literally locates things. Interesting...

C:\Users\WTCra\AppData\Local\Temp\ipykernel_18836\1165773589.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  records.loc['51 OMAHA CT', 'SACRAMENTO']


,,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
street,city,,,,,,,,,,
51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,Wed May 21 00:00:00 EDT 2008,68212,38.478902,-121.431028


In [50]:
#Now let's fund sum of transactions by city.

records[['price']].groupby(['city']).sum().sort_values(by='price', ascending=False)

,price
city,
SACRAMENTO,86806099
ELK GROVE,30911977
ROSEVILLE,15577356
EL DORADO HILLS,11309076
ANTELOPE,7672381
RANCHO CORDOVA,7375366
FOLSOM,7054323
LINCOLN,6950827
CITRUS HEIGHTS,6549022


In [51]:
#Let's also target the number of beds.

records[['price', 'beds']].groupby(['city']).sum().sort_values(by='price', ascending=False)

,price,beds
city,,
SACRAMENTO,86806099,1329
ELK GROVE,30911977,383
ROSEVILLE,15577356,133
EL DORADO HILLS,11309076,79
ANTELOPE,7672381,111
RANCHO CORDOVA,7375366,74
FOLSOM,7054323,56
LINCOLN,6950827,72
CITRUS HEIGHTS,6549022,113


In [52]:
# import ipywidgets as widgets 
#Above line used in class but not sure it's needed!

from ipywidgets import interact


In [58]:
#Let's make a slider work on the column 'price'
#Remember to use a decorator here!
@interact
def showSlider(column = 'price', x=  (0, 1000000)):
    return records.loc[records[column]>x]




interactive(children=(Text(value='price', description='column'), IntSlider(value=500000, description='x', max=…

In [60]:
records.groupby(['type']).agg({'price':'sum'})

,price
type,
Condo,8104438
Multi-Family,2918951
Residential,219333711
Unkown,275000


In [68]:
@interact
def sumGroup(column=['type','zip'], price = (0,9000000)):
    aggDF = records.groupby(['type']).agg({'price':'sum'})
    return aggDF.loc[aggDF['price'] > price]

interactive(children=(Dropdown(description='column', options=('type', 'zip'), value='type'), IntSlider(value=4…